In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import SGDClassifier as SGD
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import log_loss, f1_score

Для работы с [нейронной сетью](https://www.tensorflow.org/) сделаем соответствующие импорты

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import metrics

# Предобработка данных

Посмотрим на входные данные, при необходимости предобработаем их

In [ ]:
df_train = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/train.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2021/test.csv')
df_train.head()

In [ ]:
df_train.info()

Как мы видим, отсутствуют пустые значения. Посмотрим на сами значения

In [ ]:
print("Y unique: {}".format(np.unique(df_train['target'])))
for i in range(50):
    print("feature_{} unique: {}".format(i, np.unique(df_train['feature_{}'.format(i)])))

Как мы видим, предобработки требуют только метки классов. Предположим, что значения признаков $feature_k, k \in [0, 49]$ отсортированы по возрастанию значимости, то есть если $feature_k[i] > feature_k[j]$, то $i$-ый объект более значим по признаку $k$, чем $j$-ый объект.

In [ ]:
mapping = {
    'Class_1': 0,
    'Class_2': 1,
    'Class_3': 2,
    'Class_4': 3,
}
df_train['target'] = df_train['target'].map(mapping)
df_train['target']

In [ ]:
y_train = df_train['target']
X_train = df_train.drop(['target', 'id'], axis=1)
X_test = df_test.drop('id', axis=1)
y_sparse_train = np.zeros(4 * y_train.shape[0]).reshape((-1, 4))
for y in y_train:
    y_sparse_train[y] = 1.0

# Создание моделей

Для подбора метрик будем использовать [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

scoring = 'neg_log_loss' аналогично [log_loss](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). По условию задачи оценивается Log loss метрика, поэтому ориентироваться будем именно на нее.

Так как данных достаточно много, будем делать число фолдов, равным 10.

## [SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier)

In [ ]:
%%time
params = {
    'alpha': [0.00001, 0.0001, 0.01, 0.1],
    'early_stopping': [False, True]
}

sgd_model = SGD(loss='log', shuffle=False)
clf_SGD = GridSearchCV(sgd_model, params, scoring = 'neg_log_loss', cv=10)
clf_SGD.fit(X_train, y_train)
clf_SGD.best_params_

In [ ]:
y_pred_SGD = clf_SGD.predict_proba(X_test)
y_pred_SGD

## [SelfTrainingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.semi_supervised.SelfTrainingClassifier.html)

Классификатор основывается на удалении некоторых меток классов и последующем восстановлении их.

Для того чтобы натренировать алгоритм, добавить неизвестные метки классов, как тестовый набор.

[Статья](https://towardsdatascience.com/a-gentle-introduction-to-self-training-and-semi-supervised-learning-ceee73178b38)

In [ ]:
X_unlabeled = X_test
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_train, y_train, test_size=0.90, stratify=y_train)
y_train_c

In [ ]:
iterations = 0
train_f1s = []
test_f1s = []
pseudo_labels = []
high_prob = [1]

while len(high_prob) > 0 and X_unlabeled.shape[0] > 0:
    print("Итерация {}".format(iterations))
    # Обучаем классификатор и делаем предсказания
    clf = DecisionTreeClassifier()
    clf.fit(X_train_c, y_train_c)
    y_hat_train = clf.predict(X_train_c)
    y_hat_test = clf.predict(X_test_c)
    
    # Вычисляем f1_score
    f1s_train = f1_score(y_train_c, y_hat_train, average=None)
    f1s_test = f1_score(y_test_c, y_hat_test, average=None)
    train_f1s.append(f1s_train)
    test_f1s.append(f1s_test)
    print('Train f1: {}'.format(f1s_train))
    print('Test f1: {}'.format(f1s_test))
    print(np.unique(y_hat_train))
    print(np.unique(y_hat_test))
    
    # генерирование вероятностей
    pred_probs = clf.predict_proba(X_unlabeled)
    preds = clf.predict(X_unlabeled)
    prob_0 = pred_probs[:, 0]
    prob_1 = pred_probs[:, 1]
    prob_2 = pred_probs[:, 2]
    prob_3 = pred_probs[:, 3]
    
    # Хранение вероятностей и предсказываний в датафрейме
    df_pred_prob = pd.DataFrame([])
    df_pred_prob['preds'] = preds
    df_pred_prob['prob_0'] = prob_0
    df_pred_prob['prob_1'] = prob_1
    df_pred_prob['prob_2'] = prob_2
    df_pred_prob['prob_3'] = prob_3
    df_pred_prob.index = X_unlabeled.index
    
    # Разделяем предсказания с более чем 99,5% вероятностью
    high_prob = pd.concat([df_pred_prob.loc[df_pred_prob['prob_0'] > 0.999],
                           df_pred_prob.loc[df_pred_prob['prob_1'] > 0.999],
                          df_pred_prob.loc[df_pred_prob['prob_2'] > 0.999],
                          df_pred_prob.loc[df_pred_prob['prob_3'] > 0.999]],
                          axis=0)
    print("{} предсказаний с высокой вероятностью".format(len(high_prob)))
    pseudo_labels.append(len(high_prob))
    
    # Добавляем псевдо-метки к данным в тренировке
    X_train_c = pd.concat([X_train_c, X_unlabeled.loc[high_prob.index]], axis=0)
    y_train_c = pd.concat([y_train_c, high_prob.preds])
    
    # Убираем псевдо-метки из неизвестных данных
    X_unlabeled = X_unlabeled.drop(index=high_prob.index)
    print("Осталось {} данных без меток".format(len(X_unlabeled)))
    
    # Увеличиваем счетчик итераций
    iterations += 1
    print()

Получаем, что дерево решений не колеблется в определении меток классов - легче использовать его.

Заметим, что точность предсказаний достаточно высока

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred_STC = clf.predict_proba(X_test)
y_pred_STC

# [Keras](https://keras.io/api/)

In [ ]:
%%time
model = Sequential()
model.add(Dense(2, input_dim=X_train.shape[1], activation='softsign'))
model.add(Dense(5, activation='softsign'))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')
model.fit(X_train, y_sparse_train)

best_last_error = np.inf
best_n_neurons = 1
h_best = []

for i in range(1, 25, 3):
    model = Sequential()
    model.add(Dense(20, input_dim=X_train.shape[1], activation='softsign'))
    model.add(Dense(i, activation='softsign'))
    model.add(Dense(4, activation='softmax'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    print('Num neurons: {}'.format(i))
    %time history = model.fit(X_train, y_sparse_train, epochs=50, verbose=0).history['loss']
    print(history)
    last_error = history[-1]
    if best_last_error > last_error:
        best_last_error = last_error
        best_n_neurons = i

Как мы видим, модель обучилась достаточно хорошо - возможно, даже переобучилась

In [ ]:
model = Sequential()
model.add(Dense(20, input_dim=X_train.shape[1], activation='softsign'))
model.add(Dense(best_n_neurons, activation='softsign'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')
print('Num neurons: {}'.format(best_n_neurons))
%time history = model.fit(X_train, y_sparse_train, epochs=200, verbose=0).history['loss']
y_pred_keras = model.predict(X_test)
y_pred_keras

# Запись данных в файл

In [ ]:
X_test.info()

In [ ]:
output_SGD = pd.DataFrame({'id': df_test.id, 'Class_1': y_pred_SGD[:, 0],'Class_2': y_pred_SGD[:, 1], 'Class_3': y_pred_SGD[:, 2], 'Class_4': y_pred_SGD[:, 3]})
output_SGD.to_csv('SGDClassifier.csv', index=False)
output_SGD.info()

In [ ]:
output_SGD.head()

In [ ]:
output_STC = pd.DataFrame({'id': df_test.id, 'Class_1': y_pred_STC[:, 0],'Class_2': y_pred_STC[:, 1], 'Class_3': y_pred_STC[:, 2], 'Class_4': y_pred_STC[:, 3]})
output_STC.to_csv('SelfTrainingClassifier.csv', index=False)
output_STC.info()

In [ ]:
output_STC.head()

In [ ]:
output_nn = pd.DataFrame({'id': df_test.id, 'Class_1': y_pred_keras[:, 0],'Class_2': y_pred_keras[:, 1], 'Class_3': y_pred_keras[:, 2], 'Class_4': y_pred_keras[:, 3]})
output_nn.to_csv('keras.csv', index=False)
output_nn.info()

In [ ]:
output_nn.head()

# Вывод

Лучше всех отработал линейный SGD классификатор. Дерево явно переобучилось, а нейронная сеть отработала немного хуже